In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from math import radians, sin, cos, sqrt, atan2
# from geopy.distance import geodesic
from sklearn.cluster import KMeans
from haversine import haversine
from sklearn.model_selection import train_test_split
# import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

# 
# import folium
import zipfile


np.random.seed(42)

In [3]:
%%time
data = pd.read_csv('data/NYC_Taxi/train.csv')
data.set_index('id',inplace=True)
data.head(5)

CPU times: user 2.57 s, sys: 434 ms, total: 3.01 s
Wall time: 3.01 s


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [4]:
data.shape

(1458644, 10)

In [5]:
data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration'],
      dtype='object')

In [6]:
pd.set_option("display.float_format", "{:.2f}".format)
data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,1.66,-73.97,40.75,-73.97,40.75,959.49
std,0.50,1.31,0.07,0.03,0.07,0.04,5237.43
min,1.00,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,9.00,-61.34,51.88,-61.34,43.92,3526282.00


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1458644 entries, id2875421 to id1209952
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   vendor_id           1458644 non-null  int64  
 1   pickup_datetime     1458644 non-null  object 
 2   dropoff_datetime    1458644 non-null  object 
 3   passenger_count     1458644 non-null  int64  
 4   pickup_longitude    1458644 non-null  float64
 5   pickup_latitude     1458644 non-null  float64
 6   dropoff_longitude   1458644 non-null  float64
 7   dropoff_latitude    1458644 non-null  float64
 8   store_and_fwd_flag  1458644 non-null  object 
 9   trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 122.4+ MB


In [8]:
data = data.loc[data['trip_duration']<20000]

There are outliers here. This might not comply with our obervation. No one takes 2 million seconds in cab travelling.

In [9]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'],format='%Y-%m-%d %H:%M:%S')

In [10]:
data = data.loc[(data['pickup_latitude'] >= 40.637044) & (data['pickup_latitude'] <= 40.855256)]
data = data.loc[(data['pickup_longitude'] >= -74.035735) & (data['pickup_longitude'] <= -73.770272)]
data = data.loc[(data['dropoff_latitude'] >= 40.637044) & (data['dropoff_latitude'] <= 40.855256)]
data = data.loc[(data['dropoff_longitude'] >= -74.035735) & (data['dropoff_longitude'] <= -73.770272)]
data.shape

(1438064, 10)

If you obeserve the map, most of the pickups and dropoffs are downtown of manhattan. and you see something on the right bottom corner, its JFK airport. The dropoffs are scattered in manhattan and near manhattan. So if the dropoffs is manhattan or via manhattan then at peak time the duration will be a lot.

In [11]:
data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['day_of_week'] = data['day_of_week'].map(lambda x: day_names[x])
data['day_of_week']

id
id2875421       Monday
id2377394       Sunday
id3858529      Tuesday
id3504673    Wednesday
id2181028     Saturday
               ...    
id2376096       Friday
id1049543       Sunday
id2304944       Friday
id2714485      Tuesday
id1209952      Tuesday
Name: day_of_week, Length: 1438064, dtype: object

In [12]:
data['trip_in_minutes'] = data['trip_duration']/60
# sns.boxplot(data=data,x='day_of_week',y='trip_in_minutes')
# plt.ticklabel_format(style='plain', axis='y')
# plt.show()

There are outliers here. This might not comply with our obervation. No one takes 30000 minutes in cab travelling.

Lets see rushhour trips. Based on research

**Morning hours: 6:00 AM to 11:00 AM\
Afternoon hours: 11:00 AM to 4:00 PM\
Evening hours: 4:00 PM to 8:00 PM \
Night hours: 8:00 PM to 2:00 AM**

In [13]:
data['pickup_hour'] = data['pickup_datetime'].dt.hour
data['pickup_hour'].head()

id
id2875421    17
id2377394     0
id3858529    11
id3504673    19
id2181028    13
Name: pickup_hour, dtype: int32

In [14]:
data['pickup_month'] = data['pickup_datetime'].dt.month

# sns.countplot(data=data,x='pickup_month')

Its strange to see but there is only 6 months of data and August is starting of tourist season but this data doesnt have next 6 months.

In [15]:
data['date'] = data['pickup_datetime'].dt.day
# plt.figure(figsize=(14,8))
# sns.countplot(data=data,x='date',hue='vendor_id')

There's not a lot to observe here.

Considerbly, weekends have more trips.

We have latitudes and longitudes which we cannot use directly in modeling. So I am going to experiment those into clusters.

In [16]:
pickup_coordinates = data[['pickup_latitude','pickup_longitude']]
dropoff_coordinates = data[['dropoff_latitude','dropoff_longitude']]

n_clusters = 5

pickup_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
data['pickup_cluster_label'] = pickup_kmeans.fit_predict(pickup_coordinates)

dropoff_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
data['dropoff_cluster_label'] = dropoff_kmeans.fit_predict(dropoff_coordinates)

/home/hpc/iwi3/iwi3083h/.conda/envs/hdc/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/hpc/iwi3/iwi3083h/.conda/envs/hdc/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [17]:
data['haversine_distance'] = data.apply(lambda x: haversine((x['pickup_latitude'],x['pickup_longitude']),(x['dropoff_latitude'],x['dropoff_longitude'])),axis=1)
data['haversine_distance'].head(10)

id
id2875421   1.50
id2377394   1.81
id3858529   6.39
id3504673   1.49
id2181028   1.19
id0801584   1.10
id1813257   1.33
id1324603   5.71
id1301050   1.31
id0012891   5.12
Name: haversine_distance, dtype: float64

Thats it for understanding the data, lets model it.

## Data Modeling

In [18]:
data = pd.get_dummies(data,columns=['vendor_id','store_and_fwd_flag','day_of_week'])
data.columns

Index(['pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'trip_duration', 'trip_in_minutes', 'pickup_hour',
       'pickup_month', 'date', 'pickup_cluster_label', 'dropoff_cluster_label',
       'haversine_distance', 'vendor_id_1', 'vendor_id_2',
       'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday'],
      dtype='object')

In [19]:
data.drop(columns=["pickup_datetime", "dropoff_datetime"], inplace = True)

In [21]:
data['trip_duration'].describe()

count   1438064.00
mean        824.70
std         647.44
min           1.00
25%         394.00
50%         655.00
75%        1057.00
max       18794.00
Name: trip_duration, dtype: float64

In [22]:
data['trip_duration'] = pd.cut(data['trip_duration'], [0,400,660,1060,3000, 20000], labels = [1, 2, 3, 4, 5])

In [23]:
data['trip_duration'].value_counts()

trip_duration
1    368898
2    356504
3    355521
4    337232
5     19909
Name: count, dtype: int64

In [24]:
y = data['trip_duration']
X = data.drop(['trip_duration'],axis=1)

In [25]:
data.to_csv("NYC_cleaned.csv", index=False)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.3)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
model = RandomForestRegressor()

model.fit(X_train,y_train)

In [ ]:
importances = model.feature_importances_
feature_names = X.columns
indices = np.argsort(importances)[::-1]
sorted_feature_names = [feature_names[i] for i in indices]
sorted_importances = importances[indices]

plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_importances)), sorted_importances, align='center')
plt.xticks(range(len(sorted_importances)), sorted_feature_names, rotation='vertical')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.tight_layout()

In [ ]:
ypred = model.predict(X_test)

print("MSE: ",mean_squared_error(y_test,ypred))

In [ ]:
adamodel = AdaBoostRegressor()
adamodel.fit(X_train,y_train)

ypred = adamodel.predict(X_test)

print("MSE: ",mean_squared_error(y_test,ypred))

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_pars = {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.8, 'max_depth': 15,
            'subsample': 0.75, 'lambda': 2, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
            'eval_metric': 'rmse', 'objective': 'reg:squarederror'}    

model = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=250,
                  maximize=False, verbose_eval=15)